# Things we own database

Postgress database to keep inventory of all of the things that we own in preparation for moving.

Schema design saved on [DBDesigner](https://app.dbdesigner.net/designer/schema/401855).

In [32]:
# imports
import psycopg2
import pandas as pd
from contextlib import contextmanager
from typing import Dict, List

**Variables**

Declaring databse variables to allow for connection.

In [2]:
# Variables
database = "House"
user = "iainmcl"
password = "empire"
host = "127.0.0.1"
port = "5433"

## Context manager

Create a context manager to handle connection to the postgres database and ensure that it is closed.

In [3]:
# Database context manger
@contextmanager
def connect(database=database, user=user, password=password, host=host, port=port):
    conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)
    try:
        yield conn
    finally: 
        conn.close()

## Setup functions

- Create tables
- Execute script

In [4]:
def execute_script(cur, script):
    with open(script) as s:
        query = s.read()
        cur.execute(query)

In [5]:
def create_tables(sql_path="./scripts/HouseMoving_postgres_create.sql"):
    with connect() as conn:
        cur = conn.cursor()
        execute_script(cur, sql_path)
        conn.commit()

## Insert functions

- add_person()

In [41]:
def add_person(name:str, priority:int) -> Dict[str, int]:
    with connect() as conn:
        cur = conn.cursor()
        cur.execute(f"INSERT INTO \"Person\" values (\'{name.lower()}\', {priority.lower()});")
        conn.commit()
    return {'name': name, 'priority': priority}

In [43]:
def add_room(house: str, room: str) -> Dict[str, str]:
    with connect() as conn:
        cur = conn.cursor()
        cur.execute(f"INSERT INTO \"House\" values (\'{house.lower()} {room.lower()}\',\'{house.lower()}\', \'{room.lower()}\');")
        conn.commit()
    return {'house': house, 'room': room}

In [57]:
def add_item(name:str, room:str, owner:str, value=0, quant:int=1, size:str='medium', priority: int=0, fragile:bool=False, owned:bool=True, moved:bool=False, keeping:bool=True, notes:str=""):
    with connect() as conn:
        cur = conn.cursor()
        cur.execute(f"""
            INSERT INTO \"Item\" values(
                \'{name}\',
                {quant},
                \'{size}\',
                {priority},
                {fragile},
                {owned},
                {moved},
                {keeping},
                {value},
                \'{owner}\',
                \'{notes}\',
                \'{room.lower()}\'
                )""")
        conn.commit()
    

## Select functions

- Select all


In [95]:
def select_all(table_name="Items", outfile=False):
    with connect() as conn:
        cur = conn.cursor()
        execute_script(cur, "./scripts/select_all.sql")
        data = cur.fetchall()
    return data        

In [63]:
def get_room_options(outfile=False):
    with connect() as conn:
        cur = conn.cursor()
        cur.execute("SELECT \"HouseName\", \"Room\" FROM \"House\";")
        data = cur.fetchall()
    return data

In [65]:
def get_person_options(outfile=False):
    with connect() as conn:
        cur.conn.cursor()
        cur.execute("SELECT \"Name\" FROM PERSON;")
        data = cur.fetchall()
    return data

In [66]:
def get_items():
    with connect() as conn:
        cur = conn.cursor()
        cur.execute("SELECT \"ItemName\" FROM \"Item\";")
        data = cur.fetchall()
    return data

## Delete functions

In [81]:
def delete_person(name):
    with connect() as conn:
        cur = conn.cursor()
        cur.execute(f"DELETE FROM \"Person\" WHERE \"Name\" = \'{name}\';")
        conn.commit()
    return name

In [89]:
def delete_item(ItemName):
    with connect() as conn:
        cur = conn.cursor()
        cur.execute(f"DELETE FROM \"Item\" WHERE \"ItemName\" = \'{ItemName}\';")
        conn.commit()
    return ItemName

In [96]:
def delete_room(house_room):
    with connect() as conn:
        cur = conn.cursor()
        cur.execute(f"DELETE FROM \"House\" WHERE \"House_pk\" = \'{house_room}\';")
        conn.commit()
    return house_room

In [7]:
def drop_all():
    with connect() as conn:
        cur = conn.cursor()
        execute_script(cur, "./scripts/drop_all.sql")
        conn.commit()

In [15]:
try:
    drop_all()
except Exception as e:
    print("Table already dropped.")
    print("\t-", e)

In [21]:
create_tables()

In [38]:
add_person('Eddie', 3)

{'name': 'Eddie', 'priority': 3}

In [44]:
add_room('flat', 'living room')

{'house': 'flat', 'room': 'living room'}

In [58]:
add_item('TV', 'flat living room', 'katie')

In [90]:
delete_item('TV')

In [94]:
select_all()

[(None, None, None, None, None, None, None, None, None, None, None, None, 'katie', 5, None, None, None), (None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'flat living room', 'flat', 'living room')]


## Export functions